In [3]:

cd _scripts/

/Users/ben/prj/fab/_scripts


In [35]:

import fill_untappd
import yaml, json

In [59]:


%load_ext autoreload
%autoreload


import os
from pprint import pprint
from airtable import Airtable
import pandas as pd

base_key = 'appR312nFpbS1UbOH'
table_name = 'Cans'
airtable = Airtable(base_key, table_name, api_key="key78z9GoFTdaPlYd")
record_list = airtable.get_all()
cans_df = pd.DataFrame([record['fields'] for record in record_list]).loc[lambda x: ~x.code.isna()]




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
for i, r in cans_df.iterrows():
    s = yaml.dump(r.to_dict(), default_flow_style=False)
    with open(f"../_cans/{r.code}.md","w") as f:
        f.write("---\n")
        f.write(s)
        f.write("\n---")


In [61]:
cans_df.to_json("../_data/cans.json", orient="records")

In [62]:
%autoreload

fill_untappd.run()

{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 844837932}
{'count': 0, 'items': []}


In [63]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins limit 5;", conn)

In [64]:

venues_info

venue_id  \
venue_slug                             beer_slug                                          
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo    4367121.0   
                                       fab-fermentation-arts-squid-pro-quo    4367121.0   
NaN                                    fab-fermentation-arts-squid-pro-quo          NaN   
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo    4367121.0   
flatbread-company-at-saccos-bowl-haven fab-fermentation-arts-balloon-factory    12317.0   

                                                                                                           venue_name  \
venue_slug                             beer_slug                                                                        
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo                 Dorchester Brewing Company   
                                       fab-fermentation-arts-squid-pro-quo                 Dorchester Brewing Company   
NaN                                    fab-fermentation-arts-squid-pro-quo                                        NaN   
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo                 Dorchester Brewing Company   
flatbread-company-at-saccos-bowl-haven fab-fermentation-arts-balloon-factory  Flatbread Company at Sacco's Bowl Haven   

                                                                             primary_category_key  \
venue_slug                             beer_slug                                                    
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo         Nightlife Spot   
                                       fab-fermentation-arts-squid-pro-quo         Nightlife Spot   
NaN                                    fab-fermentation-arts-squid-pro-quo                    NaN   
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo         Nightlife Spot   
flatbread-company-at-saccos-bowl-haven fab-fermentation-arts-balloon-factory                 Food   

                                                                             primary_category  \
venue_slug                             beer_slug                                                
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo     Nightlife Spot   
                                       fab-fermentation-arts-squid-pro-quo     Nightlife Spot   
NaN                                    fab-fermentation-arts-squid-pro-quo                NaN   
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo     Nightlife Spot   
flatbread-company-at-saccos-bowl-haven fab-fermentation-arts-balloon-factory             Food   

                                                                                    parent_category_id  \
venue_slug                             beer_slug                                                         
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo    4d4b7105d754a06376d81259   
                                       fab-fermentation-arts-squid-pro-quo    4d4b7105d754a06376d81259   
NaN                                    fab-fermentation-arts-squid-pro-quo                         NaN   
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo    4d4b7105d754a06376d81259   
flatbread-company-at-saccos-bowl-haven fab-fermentation-arts-balloon-factory  4d4b7105d754a06374d81259   

                                                                                                                     categories  \
venue_slug                             beer_slug                                                                                  
dorchester-brewing-company             fab-fermentation-arts-squid-pro-quo    {'count': 2, 'items': [{'category_key': 'brewe...   
                                       fab-fermentation-arts-squid-pr

In [65]:
print(len(venues_info))
print(len(venues_info.loc[~venues_info.index.duplicated()]))


5
3


In [67]:
jsons  = df.json.apply(lambda j: json.loads(j))

venues = jsons.apply(
    lambda x: 
    pd.concat([
        pd.Series(x["venue"]),
        pd.Series(x["user"])[["user_name"]],
        pd.Series(x["beer"]),
        pd.Series(x)[["checkin_id","created_at","rating_score"]]
        ])
        )

venues_info = pd.concat([venues, 
pd.concat([
    venues['contact'].apply(lambda x: pd.Series(x)),
    venues['location'].apply(lambda x: pd.Series(x)),
], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins")

venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts],axis=1))

venues_annotated.to_csv("../_data/venues.csv")